In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import tensorflow.keras.metrics as Metrics
import tensorflow.keras.utils as Utils
from keras.utils.vis_utils import model_to_dot
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix as CM
from random import randint
from IPython.display import SVG
import matplotlib.gridspec as gridspec
import pandas as pd
from keras.applications.inception_v3 import InceptionV3
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.applications.inception_v3 import InceptionV3


In [3]:
!apt-get install -y libsm6 libxext6 libxrender-dev
!pip install opencv-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6 is already the newest version (2:1.2.2-1).
libxext6 is already the newest version (2:1.3.3-1).
libxrender-dev is already the newest version (1:0.9.10-1).
libxrender-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install tensorflow==2.6.0
!pip install keras~=2.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     | 564.4 MB 131.4 MB/s
     |████████████████████████████████| 132 kB 8.1 MB/s 
     |████████████████████████████████| 14.8 MB 53.3 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=eb10209f98f9444f51c9d1d024c12f2ae3d3f558cba0d753aec4b4f8b77f01e6
  Stored in directory: /root/.cache/pip/wheels/98/91/04/971b4c587cf47ae952b108949b46926f426c02832d120a082a
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68728 sha256=903dc85846784b92fb5d80241ad4363bd2af6fa9f81b0e8dd476cda620378ec5
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built clang wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attemptin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.2 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0


In [5]:
!pip install -U efficientnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.0 MB/s 


In [6]:
from pathlib import Path
train_path = Path("/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train")
train_path.glob(r'**/*.png')

valid_path = Path("/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/val")
valid_path.glob(r'**/*.png')

<generator object Path.glob at 0x7fd452e0f7d0>

In [7]:
train_filepath = list(train_path.glob(r'**/*.png'))
valid_filepath = list(valid_path.glob(r'**/*.png'))

print(train_filepath)

[PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00000.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00001.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00002.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00016.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00014.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00009.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00018.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00010.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dat

In [8]:
os.path.split(os.path.split(train_filepath[0])[1]) 
    
    

('', 'img00000.png')

In [9]:
len(list(map(lambda x: os.path.split(os.path.split(x)[0])[1],valid_filepath)))

133

In [10]:
len(list(map(lambda x: os.path.split(os.path.split(x)[0])[1],train_filepath)))

864

In [11]:
classes_train = []
classes_valid = []


In [14]:
csv_file = pd.read_csv('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train_data.csv')

for i in range(1000):
  cnt = 0
  for j in csv_file.file_name:
    cnt = cnt + 1
    num = int(j[3:8])

    if(i == num):
      cnt2 = 0
      for k in csv_file['class']:
        cnt2 = cnt2 + 1
        if(cnt2 == cnt):
          classes_train.append(k)
          cnt = 0
          cnt2 = 0
          break
      

  

In [15]:
csv_file = pd.read_csv('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/val_data.csv')

for i in range(1000):
  cnt = 0
  for j in csv_file.file_name:
    cnt = cnt + 1
    num = int(j[3:8])

    if(i == num):
      cnt2 = 0
      for k in csv_file['class']:
        cnt2 = cnt2 + 1
        if(cnt2 == cnt):
          classes_valid.append(k)
          cnt = 0
          cnt2 = 0
          break
      


In [16]:
print(classes_train)

['bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed'

In [18]:
print(classes_train[71])

bag


In [19]:
print(len(classes_train))

864


In [20]:
print(classes_valid)

['bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'chair', 'chair', 'chair', 'chair', 'chair', 'chair', 'chair', 'chair', 'chair', 'chair', 'chair', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'laptop', 'laptop', 'laptop', 'laptop', 'laptop', 'laptop', 'laptop', 'laptop', 'laptop', 'LivingSofa', 'LivingSofa', 'LivingSofa', 'LivingSofa', 'LivingSofa', 'LivingSofa', 'L

In [21]:
print(len(classes_valid))

133


In [22]:
train_df = pd.concat([pd.Series(train_filepath).astype(str), pd.Series(classes_train)], axis=1)
train_df.columns = ['Images', 'Image_label']

valid_df = pd.concat([pd.Series(valid_filepath).astype(str), pd.Series(classes_valid)], axis=1)
valid_df.columns = ['Images', 'Image_label']


In [23]:
print(train_df)

                                                Images Image_label
0    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
1    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
2    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
3    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
4    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
..                                                 ...         ...
859  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
860  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
861  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
862  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
863  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe

[864 rows x 2 columns]


In [24]:
print(valid_df)

                                                Images Image_label
0    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
1    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
2    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
3    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
4    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
..                                                 ...         ...
128  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
129  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
130  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
131  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
132  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe

[133 rows x 2 columns]


In [25]:
train_df.shape

(864, 2)

In [26]:
valid_df.shape

(133, 2)

In [30]:
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, 
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip =True)
test_datagen = ImageDataGenerator(rescale = 1.0/255.) 

In [31]:
train_gen = train_datagen.flow_from_dataframe(dataframe = train_df,
                                              x_col = 'Images', y_col ='Image_label',
                                              target_size = (200,200), batch_size = 512, 
                                              class_mode = 'categorical', shuffle = True)

val_gen = train_datagen.flow_from_dataframe(valid_df, 
                                            target_size=(200,200), x_col = 'Images', y_col ='Image_label', 
                                            class_mode='categorical',
                                            batch_size= 512, shuffle=True)
# test 도 있음        

Found 864 validated image filenames belonging to 11 classes.
Found 133 validated image filenames belonging to 11 classes.


In [32]:
train_gen.class_indices

{'LivingSofa': 0,
 'bag': 1,
 'bed': 2,
 'chair': 3,
 'coffeetable': 4,
 'cup': 5,
 'kitchentools': 6,
 'lamp': 7,
 'laptop': 8,
 'pot': 9,
 'shoe': 10}

In [ ]:
#!pip install q tensorflow==2.1
#!pip install q keras==2.3.1

In [35]:
inceptionV3 = InceptionV3(include_top= False, input_shape=(150,150,3))

for layer in inceptionV3.layers:
	layer.trainable = False

87924736/87910968 [==============================] - 1s 0us/step


In [36]:
last_layer = inceptionV3.get_layer('mixed9')

print('last layer output shape: ', last_layer.output_shape)

last_output = last_layer.output

last layer output shape:  (None, 3, 3, 2048)


In [38]:
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(units = 1024, activation = tf.nn.relu)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense  (6, activation = tf.nn.softmax)(x)

model = tf.keras.Model( inceptionV3.input, x)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=1,
                                            verbose=1,
                                            factor=0.25,
                                            min_lr=0.000003)

model.compile(loss = 'categorical_crossentropy', optimizer= tf.keras.optimizers.Adam(), metrics=['acc'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [40]:
history = model.fit(train_gen,
                    epochs = 10,
                    verbose = 1,
                   validation_data = val_gen,
                   callbacks=[learning_rate_reduction])

Epoch 1/10


InvalidArgumentError: ignored

In [ ]:
%matplotlib inline
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
history = model.fit(train_images, train_labels, batch_size=128, epochs=20, validation_split = 0.2)